# Assignment 4: Text Analysis

**Please insert your net id and name:**
**Monica Flores**
**mf3435**

The goal of assignment 4 is to replicate the material we covered in the Text Analysis Lab. You will be processing and analyzing a corpus of text by using different kind of techniques we learned in class. The data you will be working with contains information on social service agencies. You can find the data in the wager_ada_2018/shared folder. It is called socialservice.csv. 

Our text data table has 7 fields:

- `FACID` - unique ID for each facility.
- `facname` - name of the current facility.
- `factype` - type of the current facility.
- `facurl` - URL of facility's main web page.
- `facloc` - Location of facilitiy.
- `abouturl` - URL of "about" page used to collect text.
- `textfromurl` - Text collected from about URL.

In [64]:
# Change into correct folder. 
%cd /projects/wagner_ada_2018/shared

/projects/wagner_ada_2018/shared


1. Please load the dataset into a dataframe
2. How many facilities and types of facilities are in this dataset? Please list unique facility types and names
3. Clean and process the column `textfromurl` by executing following steps
    - Form a corpus of text
    - Clean and normalize text
4. Create a bag of words and set of features, including 
    - Removal of stop-words
    - Stemming and lemmatization
    - Adding context by N-grams
    - Add frequency weights for word counts
5. Count of the number of times that each of the words appears in the corpus
6. Generate 15 topics with the top 20 words

In [3]:
%pylab inline 
import nltk
import json
import re
import time
import progressbar
import sqlalchemy
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from __future__ import print_function
from six.moves import zip, range 
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, roc_auc_score, auc
from sklearn import preprocessing
from collections import Counter, OrderedDict
from nltk.corpus import stopwords
from nltk import SnowballStemmer

import wordcloud 
from wordcloud import WordCloud

Populating the interactive namespace from numpy and matplotlib


In [4]:
nltk.download('stopwords') #download the latest stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /nfshome/mf3435/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
#Q1 - Please load the dataset into a dataframe
socialservice=pd.read_csv('/projects/wagner_ada_2018/shared/socialservices.csv')
socialservice.tail(5)
#146 observations

,Unnamed: 0,FACID,facname,factype,facurl,facloc,abouturl,textfromurl
141,141,235,PLOWS Council on Aging,safety net,http://www.plows.org/,South-Southwest,http://www.plows.org/?page_id=3,About Us Mission Statement- PLOWS Council on ...
142,142,247,YWCA Evanston/North Shore,safety net,http://www.ywca.org/site/c.ewK0LoO8LmK6F/b.796...,North-Northwest,http://www.ywca.org/site/c.ewK0LoO8LmK6F/b.796...,Donate Connect Volunteer Who We Are YWCA is t...
143,143,247,YWCA Evanston/North Shore,safety net,http://www.ywca.org/site/c.ewK0LoO8LmK6F/b.796...,North-Northwest,http://www.ywca.org/site/c.ewK0LoO8LmK6F/b.796...,Donate Connect Volunteer Mission & Vision YWC...
144,144,247,YWCA Evanston/North Shore,safety net,http://www.ywca.org/site/c.ewK0LoO8LmK6F/b.796...,North-Northwest,http://www.ywca.org/site/c.ewK0LoO8LmK6F/b.938...,Donate Connect Volunteer General Facts About ...
145,145,247,YWCA Evanston/North Shore,safety net,http://www.ywca.org/site/c.ewK0LoO8LmK6F/b.796...,North-Northwest,http://www.ywca.org/site/apps/nlnet/content2.a...,Donate Connect Volunteer Do you know about cr...


In [34]:
#Q2 - How many facilities and types of facilities are in this dataset? 
#Please list unique facility types and names
q2_names=socialservice.groupby('facname').count()
q2_names
#48 Unique names

,Unnamed: 0,FACID,factype,facurl,facloc,abouturl,textfromurl
facname,,,,,,,
360 Youth Services,8,8,8,8,8,8,8
Arab American Action Network,2,2,2,2,2,2,2
Association House of Chicago,3,3,3,3,3,3,3
Aunt Martha’s Youth Service Center and Healthcare Network,2,2,2,2,2,2,2
"Aunt Martha’s Youth Service Center, Inc.",4,4,4,4,4,4,4
"B.U.I.L.D., Inc.",6,6,6,6,6,6,6
"Bridge Youth and Family Services, The",1,1,1,1,1,1,1
Catholic Charities of the Archdiocese of Chicago,16,16,16,16,16,16,16
Center for Economic Progress,1,1,1,1,1,1,1


In [33]:
#Q2 - Count types of social services
q2_types=socialservice.groupby('factype').count()
q2_types
#4 unique types

,Unnamed: 0,FACID,facname,facurl,facloc,abouturl,textfromurl
factype,,,,,,,
education,29,29,29,29,29,29,29
health,32,32,32,32,32,32,32
income,45,45,45,45,45,45,45
safety net,40,40,40,40,40,40,40


In [36]:
#Q3 - Clean and process the column `textfromurl` by executing following steps
# Form a corpus of text
sstext=socialservice[['textfromurl']]    
sstext=sstext.dropna()    
#Create corpus
corpus = sstext['textfromurl'].values  
corpus[0]

'Menu About Us We understand the challenges in our community. Health care, family, education, employment; trying to find the time and guidance to address every part of life can seem impossible. Association House makes it possible, all in one place \xe2\x80\x94 a place where the staff speaks your language and helps you meet your goals by providing wrap-around services for social, educational, and health programs. We offer our community a unique blend of services, and do it with a warm, family-like atmosphere. Since 1899, Association House has worked with Chicagoans who seek tools to lead more productive lives. It is one of the oldest \xe2\x80\x9csettlement houses\xe2\x80\x9d in Chicago originally designed to provide relief and guidance to new immigrants. Today, Association House is a vital resource to under-served, multicultural communities, providing collaborative programs in English and Spanish. We promote health and wellness, educational advancement, and economic empowerment. With a 

In [38]:
#Q3 - Clean and normalize text
#the regular expressions that matches all non-characters
#get rid of punctuation and make everything lowercase
RE_PREPROCESS = re.compile( r'\W+|\d+' ) 
processed_corpus = np.array( [ re.sub( RE_PREPROCESS, ' ', description ).lower() for description in corpus ] )
processed_corpus[0]

'menu about us we understand the challenges in our community health care family education employment trying to find the time and guidance to address every part of life can seem impossible association house makes it possible all in one place a place where the staff speaks your language and helps you meet your goals by providing wrap around services for social educational and health programs we offer our community a unique blend of services and do it with a warm family like atmosphere since   association house has worked with chicagoans who seek tools to lead more productive lives it is one of the oldest settlement houses in chicago originally designed to provide relief and guidance to new immigrants today association house is a vital resource to under served multicultural communities providing collaborative programs in english and spanish we promote health and wellness educational advancement and economic empowerment with a staff of over   professionals association house impacts the liv

In [46]:
#Q4 - Create a bag of words and set of features, including 
     #Removal of stop-words
     #Stemming and lemmatization
     #Adding context by N-grams
     #Add frequency weights for word counts
        
#Define create bag of words      
def create_bag_of_words( corpus,
                         NGRAM_RANGE = ( 0, 1 ),
                         stop_words = None,
                         stem = False,
                         MIN_DF = 0.05,
                         MAX_DF = 0.95,
                         USE_IDF = False ):

    """
    Turn a corpus of text into a bag-of-words.
    
    Parameters
    -----------
    corpus: ls
        test of documents in corpus    
    NGRAM_RANGE: tuple
        range of N-gram. Default (0,1)
    stop_words: ls
        list of commonly occuring words that have little semantic
        value
    stem: bool
        use a stemmer to stem words
    MIN_DF: float
       exclude words that have a frequency less than the threshold
    MAX_DF: float
        exclude words that have a frequency greater than the threshold
    
    
    Returns
    -------
    bag_of_words: scipy sparse matrix
        scipy sparse matrix of text
    features:
        ls of words
    """
    #parameters for vectorizer 
    ANALYZER = "word" #unit of features are single words rather then phrases of words 
    STRIP_ACCENTS = 'unicode'
    stemmer = nltk.SnowballStemmer("english")

    if stem:
        tokenize = lambda x: [stemmer.stem(i) for i in x.split()]
    else:
        tokenize = None
    vectorizer = CountVectorizer(analyzer=ANALYZER,
                                tokenizer=tokenize, 
                                ngram_range=NGRAM_RANGE,
                                stop_words = stop_words,
                                strip_accents=STRIP_ACCENTS,
                                min_df = MIN_DF,
                                max_df = MAX_DF)
    
    bag_of_words = vectorizer.fit_transform( corpus ) #transform our corpus is a bag of words 
    features = vectorizer.get_feature_names()

    if USE_IDF:
        NORM = None #turn on normalization flag
        SMOOTH_IDF = True #prvents division by zero errors
        SUBLINEAR_IDF = True #replace TF with 1 + log(TF)
        transformer = TfidfTransformer(norm = NORM,smooth_idf = SMOOTH_IDF,sublinear_tf = True)
        #get the bag-of-words from the vectorizer and
        #then use TFIDF to limit the tokens found throughout the text 
        tfidf = transformer.fit_transform(bag_of_words)
        
        return tfidf, features
    else:
        return bag_of_words, features

In [61]:
#Q4 - Define english stop words and stemmer
eng_stopwords = stopwords.words('english')
stemmer = SnowballStemmer("english")
#Remove stop words and get word counts, include stemming
#Add context with N-Gram and frequency weighting
processed_bag_of_words, processed_features = create_bag_of_words( processed_corpus, 
                                                                 stop_words = eng_stopwords,
                                                                 stem=True,
                                                                 NGRAM_RANGE=(0,2),
                                                                 USE_IDF = True)
processed_features

[u'abl',
 u'abus',
 u'abus support',
 u'academ',
 u'accept',
 u'access',
 u'accommod',
 u'accomplish',
 u'account',
 u'achiev',
 u'across',
 u'act',
 u'action',
 u'action current',
 u'activ',
 u'addit',
 u'address',
 u'administr',
 u'adult',
 u'adult year',
 u'advanc',
 u'advisori',
 u'advisori council',
 u'advoc',
 u'advocaci',
 u'affect',
 u'afford',
 u'age',
 u'agenc',
 u'aid',
 u'allianc',
 u'allianc board',
 u'allianc nonprofit',
 u'allow',
 u'also',
 u'alway',
 u'american',
 u'ani',
 u'annual',
 u'annual report',
 u'anoth',
 u'anoth yw',
 u'anyon',
 u'anyon need',
 u'apart',
 u'appli',
 u'april',
 u'area',
 u'around',
 u'array',
 u'art',
 u'assess',
 u'asset',
 u'assist',
 u'associ',
 u'attorney',
 u'avail',
 u'babi',
 u'back',
 u'background',
 u'base',
 u'base valu',
 u'basic',
 u'basic money',
 u'becam',
 u'becaus',
 u'becom',
 u'began',
 u'begin',
 u'believ',
 u'believ everi',
 u'benefit',
 u'best',
 u'better',
 u'bilingu',
 u'board',
 u'board director',
 u'board login',
 u'bo

In [ ]:
#Q5- Define get word counts
def get_word_counts( bag_of_words, feature_names ):

    """
    Get the ordered word counts from a bag_of_words
    
    Parameters
    ----------
    bag_of_words: obj
        scipy sparse matrix from CounterVectorizer
    feature_names: ls
        list of words
        
    Returns
    -------
    word_counts: dict
        Dictionary of word counts
    """

    # convert bag of words to array
    np_bag_of_words = bag_of_words.toarray()
    
    # calculate word count.
    word_count = np.sum(np_bag_of_words,axis=0)
    
    # convert to flattened array.
    np_word_count = np.asarray(word_count).ravel()
    
    # create dict of words mapped to count of occurrences of each word.
    dict_word_counts = dict( zip(feature_names, np_word_count) )
    
    # Create ordered dictionary
    orddict_word_counts = OrderedDict( sorted(dict_word_counts.items(), key=lambda x: x[1], reverse=True), )
    
    return orddict_word_counts

In [62]:
#Q5 - Count of the number of times that each of the words appears in the corpus
dict_processed_word_counts = get_word_counts( processed_bag_of_words, processed_features )
dict_processed_word_counts

OrderedDict([(u'servic', 320.69236474429334),
             (u'communiti', 260.15877208852265),
             (u'famili', 249.63033237795852),
             (u'provid', 239.28540359735558),
             (u'program', 239.15937598186497),
             (u'support', 231.4234885627119),
             (u'mission', 228.74023567095625),
             (u'us', 223.57637786223765),
             (u'youth', 219.90928365256568),
             (u'chicago', 210.11924890964903),
             (u'hous', 198.73521234331113),
             (u'need', 197.79340171942732),
             (u'help', 186.69914570055528),
             (u'year', 178.78713921673344),
             (u'center', 177.62077610910944),
             (u'serv', 177.22555143116909),
             (u'live', 176.93273405168529),
             (u'event', 172.31900244440567),
             (u'care', 171.08015940911397),
             (u'peopl', 168.16608925758791),
             (u'donat', 166.78170175659633),
             (u'develop', 165.09404312221469),
   

In [55]:
#Q6 - Define create topics
def create_topics(tfidf, features, N_TOPICS=3, N_TOP_WORDS=5,):
    """
    Given a matrix of features of text data generate topics
    
    Parameters
    -----------
    tfidf: scipy sparse matrix
        sparse matrix of text features
    N_TOPICS: int
        number of topics (default 10)
    N_TOP_WORDS: int
        number of top words to display in each topic (default 10)
        
    Returns
    -------
    ls_keywords: ls
        list of keywords for each topics
    doctopic: array
        numpy array with percentages of topic that fit each category
    N_TOPICS: int
        number of assumed topics
    N_TOP_WORDS: int
        Number of top words in a given topic. 
    """
    
    with progressbar.ProgressBar(max_value=progressbar.UnknownLength) as bar:
        i=0
        lda = LatentDirichletAllocation( n_topics= N_TOPICS,
                                       learning_method='online') #create an object that will create 5 topics
        bar.update(i)
        i+=1
        doctopic = lda.fit_transform( tfidf )
        bar.update(i)
        i+=1
        
        ls_keywords = []
        for i,topic in enumerate(lda.components_):
            word_idx = np.argsort(topic)[::-1][:N_TOP_WORDS]
            keywords = ', '.join( features[i] for i in word_idx)
            ls_keywords.append(keywords)
            print(i, keywords)
            bar.update(i)
            i+=1
            
    return ls_keywords, doctopic

In [63]:
#Q6 - Generate 15 topics with the top 20 words
# create topics with stemmed words.
processed_keywords, processed_doctopic = create_topics(processed_bag_of_words, 
                                                       processed_features, 
                                                       N_TOPICS = 15,
                                                       N_TOP_WORDS= 20)

| 14 Elapsed Time: 0:00:00                                                     


0 lutheran, illinoi, child, god, famili communiti, love, faith, respons, state, neighbor, improv, intak, famili, referr, engag, central, across, il phone, adult, servic
1 neighbor, resourc center, dupag, volunt, resourc, dupag counti, center, qualiti, time, basic, meet, search, help, peopl, illinoi, opportun, year, communiti, resid, person
2 napervill, valu, dupag, digniti, communiti, day, serv, peopl, core valu, food, compass, mission valu, self suffici, suffici, servic, mission, core, high, stay connect, receiv
3 ywca, evanston, north shore, shore, ywca evanston, evanston north, north, yw, women, domest violenc, domest, get, church street, street evanston, empow, crisi, safe, relationship abus, hour domest, take action
4 cathol, chariti, cathol chariti, privaci, privaci polici, heal, without, heartland, heartland allianc, allianc, need provid, term use, direct, collect, net, polici, full potenti, need, background, potenti
5 park, survivor, contribut, tax, pleas, donat, agenc, center,